In [20]:
import torch
import tiktoken

In [21]:
tokenize = tiktoken.get_encoding("gpt2")
text = "Your journey starts with one step"
vocab_size = tokenize.n_vocab
max_size = 3

In [22]:
text_encode = tokenize.encode(text)
input_id = torch.tensor(text_encode)
text_size = len(text_encode)

In [23]:
from torch.utils.data import Dataset, DataLoader
class GPTDatasetV1(Dataset):
    def __init__(self,txt,tokenize,max_length,stride):
        self.input_ids = []
        self.target_ids = []
        
        token_ids = tokenize.encode(txt)
        print("token length : ",len(token_ids))
        
        for i in range(0,len(token_ids)-max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1 : i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index],self.target_ids[index]

In [40]:
def dataloaderV1(txt,max_length,stride,batch_size):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt,tokenizer,max_length,stride)
    dataload = DataLoader(dataset,
                        batch_size=batch_size)
    return dataload

dataload = dataloaderV1(text,3,1,1)
iterator = iter(dataload)
for batch in iterator:
    print(batch)

token length :  6
[tensor([[7120, 7002, 4940]]), tensor([[7002, 4940,  351]])]
[tensor([[7002, 4940,  351]]), tensor([[4940,  351,  530]])]
[tensor([[4940,  351,  530]]), tensor([[ 351,  530, 2239]])]
